--- Day 8: Haunted Wasteland ---

You're still riding a camel across Desert Island when you spot a sandstorm quickly approaching. When you turn to warn the Elf, she disappears before your eyes! To be fair, she had just finished warning you about ghosts a few minutes ago.

One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of network of labeled nodes.

It seems like you're meant to use the left/right instructions to navigate the network. Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!

After examining the maps for a bit, two nodes stick out: AAA and ZZZ. You feel like AAA is where you are now, and you have to follow the left/right instructions until you reach ZZZ.

This format defines each node of the network individually. For example:

RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)

Starting with AAA, you need to look up the next element based on the next left/right instruction in your input. In this example, start with AAA and go right (R) by choosing the right element of AAA, CCC. Then, L means to choose the left element of CCC, ZZZ. By following the left/right instructions, you reach ZZZ in 2 steps.

Of course, you might not find ZZZ right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary: RL really means RLRLRLRLRLRLRLRL... and so on. For example, here is a situation that takes 6 steps to reach ZZZ:

LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)

Starting at AAA, follow the left/right instructions. How many steps are required to reach ZZZ?

In [1]:
from dataclasses import dataclass
from __future__ import annotations
from math import lcm


@dataclass
class Maze:
    instruction: str
    node_map: dict[str, dict[str, str]]

    @classmethod
    def from_str(cls, input_str: str) -> Maze:
        input_lines_list = list(filter(None, input_str.splitlines()))
        instruction = input_lines_list.pop(0)
        node_map = {}
        for input_line in input_lines_list:
            node_key, node_left, node_right = input_line.replace("=", "").replace("(", "").replace(")", "").replace(",", "").split()
            node_map[node_key] = {"L": node_left, "R": node_right}

        return cls(instruction=instruction, node_map=node_map)

    @property
    def solution_steps(self) -> int:
        return self.calculate_steps("AAA")


    @property
    def ghost_solution_steps(self) -> int:
        # gonna take reddit advice and just use lcm assumption rather than complete solution
        ghost_starts = list(filter(lambda x: x[-1] == "A", self.node_map.keys()))
        return lcm(*[self.calculate_steps(x, ghost_logic=True) for x in ghost_starts])


    def calculate_steps(self, starting_node: str, ghost_logic: bool = False) -> int:
        step_counter = 0
        solved = False
        current_node = starting_node
        instruction_index = 0

        while not solved:
            current_node = self.node_map[current_node][self.instruction[instruction_index]]
            step_counter += 1
            instruction_index += 1
            if instruction_index == len(self.instruction):
                instruction_index = 0
            
            if ghost_logic:
                solved = current_node[-1] == "Z"
            else:
                solved = current_node == "ZZZ"

            # protect from infinite loop
            if step_counter > 10e10:
                return Exception("Exceed loop counter limit")


        return step_counter


In [2]:
test_input = """
RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
"""

In [3]:
test_maze = Maze.from_str(test_input)

test_maze

Maze(instruction='RL', node_map={'AAA': {'L': 'BBB', 'R': 'CCC'}, 'BBB': {'L': 'DDD', 'R': 'EEE'}, 'CCC': {'L': 'ZZZ', 'R': 'GGG'}, 'DDD': {'L': 'DDD', 'R': 'DDD'}, 'EEE': {'L': 'EEE', 'R': 'EEE'}, 'GGG': {'L': 'GGG', 'R': 'GGG'}, 'ZZZ': {'L': 'ZZZ', 'R': 'ZZZ'}})

In [4]:
assert test_maze.solution_steps == 2

In [5]:
test_input_2 = """
LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)
"""


In [6]:
test_maze_2 = Maze.from_str(test_input_2)
assert test_maze_2.solution_steps == 6

In [7]:
with open("../inputs/day8_input.txt") as f:
    input_str = f.read()

In [8]:
maze = Maze.from_str(input_str)

In [9]:
maze.solution_steps

19783

# Part 2

--- Part Two ---
The sandstorm is upon you and you aren't any closer to escaping the wasteland. You had the camel follow the instructions, but you've barely left your starting position. It's going to take significantly more steps to escape!

What if the map isn't for people - what if the map is for ghosts? Are ghosts even bound by the laws of spacetime? Only one way to find out.

After examining the maps a bit longer, your attention is drawn to a curious fact: the number of nodes with names ending in A is equal to the number ending in Z! If you were a ghost, you'd probably just start at every node that ends with A and follow all of the paths at the same time until they all simultaneously end up at nodes that end with Z.

For example:

LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)

Here, there are two starting nodes, 11A and 22A (because they both end with A). As you follow each left/right instruction, use that instruction to simultaneously navigate away from both nodes you're currently on. Repeat this process until all of the nodes you're currently on end with Z. (If only some of the nodes you're on end with Z, they act like any other node and you continue as normal.) In this example, you would proceed as follows:

Step 0: You are at 11A and 22A.
Step 1: You choose all of the left paths, leading you to 11B and 22B.
Step 2: You choose all of the right paths, leading you to 11Z and 22C.
Step 3: You choose all of the left paths, leading you to 11B and 22Z.
Step 4: You choose all of the right paths, leading you to 11Z and 22B.
Step 5: You choose all of the left paths, leading you to 11B and 22C.
Step 6: You choose all of the right paths, leading you to 11Z and 22Z.

So, in this example, you end up entirely on nodes that end in Z after 6 steps.

Simultaneously start on every node that ends with A. How many steps does it take before you're only on nodes that end with Z?

In [10]:
test_input_3 = """
LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
"""

In [11]:
test_maze_3 = Maze.from_str(test_input_3)

assert test_maze_3.ghost_solution_steps == 6

In [12]:
maze.ghost_solution_steps

9177460370549